
FOR Test
​
*# Import helpful libraries*
​


In [ ]:
# Familiar imports
import numpy as np
import pandas as pd
import seaborn as sns

# For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

# For training random forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


*# *Load the data**

In [ ]:
# Load the training data
train = pd.read_csv("../input/30dayofml/train.csv", index_col=0)
test = pd.read_csv("../input/30dayofml/test.csv", index_col=0)
df = pd.read_csv("../input/30dayofml/train.csv")
df_test = pd.read_csv("../input/30dayofml/test.csv")
sample_submission = pd.read_csv("../input/30dayofml/sample_submission.csv")
X2 = pd.read_csv('../input/30dayofml/train.csv',encoding='utf-8',index_col=0)
test2 = pd.read_csv('../input/30dayofml/test.csv',encoding='utf-8',index_col=0)

y2 = X2['target']
X2 = X2.drop(['target'], axis= 1)

# Preview the data
train.head()

In [ ]:
# Separate target from features
y = train['target']
features = train.drop(['target'], axis=1)

# Preview features
features.head()

#Finding the missing values

In [ ]:
train.isnull().sum()
test.isnull().sum()

#SNS

In [ ]:
sns.countplot(x='cat0',data=train)

In [ ]:
sns.countplot(x='cat1',data=train)

In [ ]:
sns.countplot(x='cat2',data=train)

In [ ]:
sns.countplot(x='cat3',data=train)

In [ ]:
sns.countplot(x='cat4',data=train)

In [ ]:
sns.countplot(x='cat5',data=train)

In [ ]:
sns.countplot(x='cat6',data=train)

In [ ]:
sns.countplot(x='cat7',data=train)

In [ ]:
sns.countplot(x='cat8',data=train)

In [ ]:
sns.countplot(x='cat9',data=train)

In [ ]:
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0)

In [ ]:
# Define the model 
model = RandomForestRegressor(random_state=1)

# Train the model (will take about 10 minutes to run)
model.fit(X_train, y_train)
preds_valid = model.predict(X_valid)
print(mean_squared_error(y_valid, preds_valid, squared=False))

In [ ]:
# Use the model to generate predictions
predictions = model.predict(X_test)

# Save the predictions to a CSV file
output = pd.DataFrame({'Id': X_test.index,
                       'target': predictions})
output.to_csv('submission.csv', index=False)

In [ ]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_absolute_error
X_test = np.ascontiguousarray(X_test)
preds = my_model.predict(X_test)
predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

In [ ]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             #early_stopping_rounds=5, 
             early_stopping_rounds=20, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

In [ ]:
# Use the model to generate predictions
predictions = my_model.predict(X_test)

# Save the predictions to a CSV file
output = pd.DataFrame({'Id': sample_submission.id,
                       'target':predictions})
output.to_csv('XGBRegressorsubmission.csv', index=False)

#LGBM

In [ ]:
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [ ]:
label = LabelEncoder()
categorical_feature = np.where(X2.dtypes != 'float64')[0].tolist()
categorical_feature_columns = X2.select_dtypes(exclude=['float64']).columns

for column in categorical_feature_columns:
        label.fit(X2[column])
        X2[column] = label.transform(X2[column])
        test2[column] = label.transform(test2[column])

In [ ]:
'''lgbm_parameters = {
    'metric': 'rmse', 
    'n_jobs': -1,
    'n_estimators': 50000,
    'reg_alpha': 10.924491968127692,
    'reg_lambda': 17.396730654687218,
    'colsample_bytree': 0.21497646795452627,
    'subsample': 0.7582562557431147,
    'learning_rate': 0.009985133666265425,
    'max_depth': 18,
    'num_leaves': 63,
    'min_child_samples': 27,
    'max_bin': 523,
    'cat_l2': 0.025083670064082797
}
'''


In [ ]:
lgbm_parameters = {
    'metric': 'rmse', 
    'n_jobs': -1,
    'n_estimators': 70000,
    'reg_alpha': 10.924491968127692,
    'reg_lambda': 17.396730654687218,
    'colsample_bytree': 0.21497646795452627,
    'subsample': 0.7582562557431147,
    'learning_rate': 0.009985133666265425681,
    'max_depth': 20,
    'num_leaves': 63,
    'min_child_samples': 30,
    'max_bin': 523,
    'cat_l2': 0.025083670064082797,
    'feature_fraction':0.1,
    'boosting':'dart'
    }



In [ ]:
lgbm_val_pred = np.zeros(len(y2))
lgbm_test2_pred = np.zeros(len(test2))
mse = []
kf = KFold(n_splits=10, shuffle=True)

for trn_idx, val_idx in tqdm(kf.split(X2,y2)):
    x2_train_idx = X2.iloc[trn_idx]
    y2_train_idx = y2.iloc[trn_idx]
    x2_valid_idx = X2.iloc[val_idx]
    y2_valid_idx = y2.iloc[val_idx]

    lgbm_model = LGBMRegressor(**lgbm_parameters)
    lgbm_model.fit(x2_train_idx, y2_train_idx, eval_set = ((x2_valid_idx,y2_valid_idx)),verbose = -1, early_stopping_rounds = 500,categorical_feature=categorical_feature)  
    #lgbm_test2_pred += lgbm_model.predict(test2)/10
    lgbm_test2_pred += lgbm_model.predict(test2)/20
    mse.append(mean_squared_error(y2_valid_idx, lgbm_model.predict(x2_valid_idx)))
    
np.mean(mse)
pd.DataFrame({'id':test2.index,'target':lgbm_test2_pred}).to_csv('LGBMsubmission.csv', index=False)

In [ ]:
# Save the predictions to a CSV file
output = pd.DataFrame({'id':test2.index,'target':lgbm_test2_pred})
output.to_csv('LGBMsubmission3.csv', index=False)

#Thank you very much for the dedication and hard work of the Kaggle.com team.